# SVD简介
* **SVD（*Singular Value Decomposition*）奇异值分解**

SVD是一种强大的降维工具，利用SVD来逼近矩阵并从中提取重要特征。通过保留80%-90%能量，就可以得到重要的特征并去除噪声。

我们称利用SVD的方法为`隐性语义索引`（*Latent Semantic Indexing*）或者`隐性语义分析`（*Latent Semantic Analysis*）

在LSI中，一个矩阵由文档和词语组成，在矩阵上应用SVD时，产生的奇异值代表了文档的概念或主题。

## SVD应用：推荐系统
一般的方法为：我们先利SVD从数据中构建一个主题空间，然后再在该空间下计算其相似度。

**SVD的分解过程**
$$\begin{aligned}
Data_{m*n} = U_{m*m}\Sigma_{m*n}V_{n*n}^T
\end{aligned}$$

其中，矩阵$\Sigma$是奇异值矩阵，奇异值是矩阵$Data * Data^T$特征值的平方根。

## 基于协同过滤的推荐引擎
**协调过滤（*collaborative filtering*）是通过将用户和其他用户的数据进行对比来实现推荐的算法。当知道两个用户之间或两个物品之间的相似度，我们就可以利用已有的数据来预测未知的用户喜好。**

## 构建推荐系统面临的挑战
* SVD在更大规模的数据上，会降低程序的速度。在大型系统中，SVD分解会降低程序的运行速度，而且要离线运行。
* 冷启动问题（cold-start）：在协同过滤场景下，新物品的到来却反所有用户对其的喜好信息，因此无法给出推荐。解决的方法有基于内容的推荐（content-based），通过各种标签标记物品的属性，将推荐问题看成搜索问题。

奇异值的计算

In [1]:
from numpy import *
from numpy import linalg as la
U, Sigma, VT = la.svd([[1, 1], [7, 7]])

In [2]:
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

Sigma是一个矩阵，此处仅返回对角元素

In [3]:
Sigma

array([1.00000000e+01, 2.82797782e-16])

In [4]:
VT

array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678]])

### 相似度计算
* 欧式距离
* 皮尔逊相关系数
* 余弦相似度

In [5]:
def euclid_sim(in_a, in_b):
    return 1.0 / (1.0 + la.norm(in_a - in_b))


def pearson_sim(in_a, in_b):
    if (len(in_a) < 3):
        return 1.0
    return (corrcoef(in_a, in_b, rowvar=0)[0][1] + 1.0) / 2.0


def cos_sim(in_a, in_b):
    num = float(in_a.T * in_b)
    denom = la.norm(in_a) * la.norm(in_b)
    return ((num / denom) + 1.0) / 2.0

In [6]:
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]

In [7]:
mymat = mat(loadExData())

欧式距离

In [8]:
euclid_sim(mymat[:, 0], mymat[:, 4])

0.12973190755680383

In [9]:
euclid_sim(mymat[:, 0], mymat[:, 0])

1.0

余弦相似度

In [10]:
cos_sim(mymat[:, 0], mymat[:, 4])

0.5

In [11]:
cos_sim(mymat[:, 0], mymat[:, 0])

1.0

皮尔逊相关系数

In [12]:
pearson_sim(mymat[:, 0], mymat[:, 4])

0.20596538173840329

In [13]:
pearson_sim(mymat[:, 0], mymat[:, 0])

1.0

## 示例：推荐引擎
推荐系统的工作过程是:

    寻找用户没有评级的物品，即用户-物品矩阵中的0值
    遍历没有评级的物品，预测一个评级分数
    对这些物品评级进行排序，返回前N个物品
    
预测评分的伪代码为：

    遍历该用户未评过分的物品
       寻找两个用户都评过分的物品进行比较，算出相似度

In [14]:
def stand_est(datamat, user, sim_mean, item):
    n = shape(datamat)[1]
    sim_total = 0.0
    rat_sim_total = 0.0
    print('item:',item)
    for j in range(n):
        user_rating = datamat[user, j]
        if user_rating == 0:
            continue
        overlap = nonzero(logical_and(datamat[:, item].A > 0, datamat[:, j].A > 0))[0]
        print('j:',j)
        if len(overlap) == 0:
            similarity = 0
        else:
            print('datamat[overlap, item]\n',datamat[overlap, item])
            print('datamat[overlap, j]\n',datamat[overlap, j])
            similarity = sim_mean(datamat[overlap, item], datamat[overlap, j])
            sim_total += similarity
            rat_sim_total += similarity * user_rating
    if sim_total == 0:
        return 0
    return rat_sim_total / sim_total

In [15]:
def recommend(datamat, user, N=3, sim_mean=cos_sim, est_method=stand_est):
    unrated_items = nonzero(datamat[user, :].A == 0)[1]
    if len(unrated_items) == 0:
        return 'you rated everything'
    item_score = []
    for item in unrated_items:
        est_score = est_method(datamat, user, sim_mean, item)
        item_score.append([item, est_score])
    return sorted(item_score, key=lambda p:p[1], reverse=True)[:N]

In [16]:
myMat=mat(loadExData())
myMat[0,1]=myMat[0,0]=myMat[1,0]=myMat[2,0]=4
myMat[3,3]=2
myMat

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [5, 5, 5, 0, 0],
        [1, 1, 1, 0, 0]])

In [17]:
recommend(myMat, 2)

item: 1
j: 0
datamat[overlap, item]
 [[4]
 [1]
 [2]
 [5]
 [1]]
datamat[overlap, j]
 [[4]
 [1]
 [2]
 [5]
 [1]]
j: 3
datamat[overlap, item]
 [[4]
 [1]]
datamat[overlap, j]
 [[2]
 [2]]
j: 4
datamat[overlap, item]
 [[4]]
datamat[overlap, j]
 [[2]]
item: 2
j: 0
datamat[overlap, item]
 [[1]
 [2]
 [5]
 [1]]
datamat[overlap, j]
 [[1]
 [2]
 [5]
 [1]]
j: 3
datamat[overlap, item]
 [[1]]
datamat[overlap, j]
 [[2]]
j: 4


[[2, 2.5], [1, 2.0243290220056256]]

In [18]:
recommend(myMat, 2, sim_mean=euclid_sim)

item: 1
j: 0
datamat[overlap, item]
 [[4]
 [1]
 [2]
 [5]
 [1]]
datamat[overlap, j]
 [[4]
 [1]
 [2]
 [5]
 [1]]
j: 3
datamat[overlap, item]
 [[4]
 [1]]
datamat[overlap, j]
 [[2]
 [2]]
j: 4
datamat[overlap, item]
 [[4]]
datamat[overlap, j]
 [[2]]
item: 2
j: 0
datamat[overlap, item]
 [[1]
 [2]
 [5]
 [1]]
datamat[overlap, j]
 [[1]
 [2]
 [5]
 [1]]
j: 3
datamat[overlap, item]
 [[1]]
datamat[overlap, j]
 [[2]]
j: 4


[[2, 3.0], [1, 2.8266504712098603]]

In [19]:
recommend(myMat, 2, sim_mean=pearson_sim)

item: 1
j: 0
datamat[overlap, item]
 [[4]
 [1]
 [2]
 [5]
 [1]]
datamat[overlap, j]
 [[4]
 [1]
 [2]
 [5]
 [1]]
j: 3
datamat[overlap, item]
 [[4]
 [1]]
datamat[overlap, j]
 [[2]
 [2]]
j: 4
datamat[overlap, item]
 [[4]]
datamat[overlap, j]
 [[2]]
item: 2
j: 0
datamat[overlap, item]
 [[1]
 [2]
 [5]
 [1]]
datamat[overlap, j]
 [[1]
 [2]
 [5]
 [1]]
j: 3
datamat[overlap, item]
 [[1]]
datamat[overlap, j]
 [[2]]
j: 4


[[2, 2.5], [1, 2.0]]

### 基于SVD的评分估计

In [20]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [21]:
U, Sigma, VT = linalg.svd(mat(loadExData2()))
Sigma

array([15.77075346, 11.40670395, 11.03044558,  4.84639758,  3.09292055,
        2.58097379,  1.00413543,  0.72817072,  0.43800353,  0.22082113,
        0.07367823])

In [22]:
Sig2 = Sigma ** 2
sum(Sig2)

541.9999999999995

In [23]:
sum(Sig2) * 0.9

487.7999999999996

In [24]:
sum(Sig2[:2]) 

378.8295595113579

In [25]:
sum(Sig2[:3])

500.5002891275793

In [26]:
def svd_est(datamat, user, sim_meas, item):
    n = shape(datamat)[1]
    sim_total = 0.0
    rat_sim_total = 0.0
    U, Sigma, VT = la.svd(datamat)
    Sig4 = mat(eye(4) * Sigma[:4])
    x_formed_items = datamat.T * U[:, :4] * Sig4.I
    for j in range(n):
        user_rating = datamat[user, j]
        if user_rating == 0 or j == item:
            continue
        similarity = sim_meas(x_formed_items[item, :].T, x_formed_items[j, :].T)
        print('the %d and %d similarity is: %f ' %(item, j, similarity))
        sim_total += similarity
        rat_sim_total += similarity * user_rating
    if sim_total == 0:
        return 0
    else:
        return rat_sim_total / sim_total

In [27]:
recommend(mat(loadExData2()), 1, est_method=svd_est)

the 0 and 3 similarity is: 0.490950 
the 0 and 5 similarity is: 0.484274 
the 0 and 10 similarity is: 0.512755 
the 1 and 3 similarity is: 0.491294 
the 1 and 5 similarity is: 0.481516 
the 1 and 10 similarity is: 0.509709 
the 2 and 3 similarity is: 0.491573 
the 2 and 5 similarity is: 0.482346 
the 2 and 10 similarity is: 0.510584 
the 4 and 3 similarity is: 0.450495 
the 4 and 5 similarity is: 0.506795 
the 4 and 10 similarity is: 0.512896 
the 6 and 3 similarity is: 0.743699 
the 6 and 5 similarity is: 0.468366 
the 6 and 10 similarity is: 0.439465 
the 7 and 3 similarity is: 0.482175 
the 7 and 5 similarity is: 0.494716 
the 7 and 10 similarity is: 0.524970 
the 8 and 3 similarity is: 0.491307 
the 8 and 5 similarity is: 0.491228 
the 8 and 10 similarity is: 0.520290 
the 9 and 3 similarity is: 0.522379 
the 9 and 5 similarity is: 0.496130 
the 9 and 10 similarity is: 0.493617 


[[4, 3.344714938469228], [7, 3.3294020724526967], [9, 3.3281008763900686]]

## 示例：基于SVD的图像压缩

In [28]:
def print_mat(inmat, thresh=0.8):
    for i in range(32):
        for k in range(32):
            if float(inmat[i, k]) > thresh:
                print(1, end='')
            else:
                print(0, end='')
        print('')

In [29]:
def img_compress(num_sv=3, thresh=0.8):
    my1 = []
    for line in open('0_5.txt').readlines():
        new_row = []
        for i in range(32):
            new_row.append(int(line[i]))
        my1.append(new_row)
    myMat = mat(my1)
    print('****original matrix****')
    print_mat(myMat, thresh)
    U, Sigma, VT = la.svd(myMat)
    Sig_recon = mat(zeros((num_sv, num_sv)))
    for k in range(num_sv):
        Sig_recon[k, k] = Sigma[k]
    recon_mat = U[:, :num_sv] * Sig_recon * VT[:num_sv, :]
    print('****reconstructed matrix using %d singular values****' %num_sv)
    print_mat(recon_mat,thresh)

In [30]:
img_compress(2)

****original matrix****
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
00000000001111111111111110000000
0000000000011111111